In [1]:
import qdrant_client
import os, nest_asyncio
from IPython.display import Markdown, display
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import StorageContext, Document, VectorStoreIndex, SimpleDirectoryReader
os.chdir('/Users/1zuu/Library/Mobile Documents/com~apple~CloudDocs/ML DEV/LLM RESEARCH 2024/LLM in Production/01-diy-llm-qa-bot')
nest_asyncio.apply() 

In [2]:
from util.llm_services import *

Using Groq LLM ....
Using HuggingFace Embedding ....


In [4]:
documents = SimpleDirectoryReader(path_config.kb_documents_dir).load_data()
documents

[Document(id_='a23e2211-ca9f-4fe1-ac9c-fffa8a059d8f', embedding=None, metadata={'page_label': '1', 'file_name': 'HNB.pdf', 'file_path': '/Users/1zuu/Library/Mobile Documents/com~apple~CloudDocs/ML DEV/LLM RESEARCH 2024/LLM in Production/01-diy-llm-qa-bot/db/kb/HNB.pdf', 'file_type': 'application/pdf', 'file_size': 2653957, 'creation_date': '2024-05-31', 'last_modified_date': '2024-05-31'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='FIXED DEPOSIT SERVICES\nCIB/CMB User Information ManualCorporate Internet Banking', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='ed6c1717-1fa9-433e-a9d1-ab7520f1561b', embedding=None, metadata={'page_label'

In [5]:
# token_count_hnb = len(tiktoken_encoder.encode(document_hnb))
# token_count_peoples = len(tiktoken_encoder.encode(document_peoples))

# print(f"Total token count HNB: {token_count_hnb}")
# print(f"Total token count Peoples Bank: {token_count_peoples}")

In [6]:
nodes = Settings.node_parser.get_nodes_from_documents(documents)
print(f"Total nodes: {len(nodes)}")

Total nodes: 16


In [7]:
collection_name = "banks"
client = qdrant_client.QdrantClient(
                                path=path_config.vector_store_path
                                )
vector_store = QdrantVectorStore(
                                client=client, 
                                collection_name=collection_name
                                )
if not os.path.exists(f"{path_config.vector_store_path}/collection/{collection_name}"):
    print("Building Finance Index ...")
    storage_context = StorageContext.from_defaults(
                                                    vector_store=vector_store,
                                                    )
    index = VectorStoreIndex(
                            nodes, 
                            storage_context=storage_context
                            )
    # index.storage_context.vector_store.persist(persist_path=path_config.vector_store_path)
    
else:
    print("Loading Finance Index ...")
    storage_context = StorageContext.from_defaults(
                                                    vector_store=vector_store
                                                    )
    index = VectorStoreIndex.from_vector_store(
                                                vector_store, 
                                                storage_context=storage_context
                                                )
chat_engine = index.as_chat_engine()

Loading Finance Index ...


In [8]:
question = "What are the Eligibility criteria to open a Fixed Deposit?"

In [9]:
response = chat_engine.chat(question)
display(Markdown(str(response)))

The eligibility criteria to open a Fixed Deposit include being a citizen of Sri Lanka above 18 years of age holding a valid National Identity Card (NIC), a valid Passport, or a Driving License. Sri Lankan citizens residing temporarily outside Sri Lanka, individuals holding dual citizenship, and non-residents who possess a resident visa are also eligible.